In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE = 28
BATCH_SIZE = 32
EPOCHS = 10
CHANNELS = 1

In [ ]:
trainSet = tf.keras.preprocessing.image_dataset_from_directory(
    "mnist_png//training",
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="grayscale"
)

# Load testing data
testSet = tf.keras.preprocessing.image_dataset_from_directory(
    "mnist_png//testing",
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="grayscale"
)

In [ ]:
class_names = trainSet.class_names

In [ ]:
def get_dataset_partition(trainSet, trainSplit=0.9, valSplit=0.1, shuffle=True, shuffleSize=10000):
    ds_size = len(trainSet)
    train_size = int(ds_size * trainSplit)
    val_size = int(ds_size * valSplit)

    train_ds = trainSet.take(train_size)
    val_ds = trainSet.skip(train_size).skip(val_size)
    return train_ds, val_ds

In [ ]:
train_ds, val_ds = get_dataset_partition(trainSet)

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = testSet.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resizeAndRescale = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1.0/255),
    layers.Reshape((IMG_SIZE, IMG_SIZE, CHANNELS)),
])

In [ ]:
dataAugmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),
])

In [ ]:
input_shape = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)
model = models.Sequential([
    resizeAndRescale,
    dataAugmentation,
    layers.Conv2D(32, 3, activation="relu", input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax"),
])

model.build(input_shape=(None, IMG_SIZE, IMG_SIZE, CHANNELS))

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='lower right')
plt.title('Training and Validation Loss')